In [1]:
import pandas as pd
from tqdm.autonotebook import tqdm
from rapidfuzz import fuzz
from rapidfuzz import process
import multiprocessing
import itertools
from scipy import sparse

data = "920"

/scratch/slurm_tmpdir/job_23803810/ipykernel_348609/3729085090.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
df = pd.read_parquet(f"../../preds{data}/REL_output/preds.pqt")

In [3]:
vertices = pd.read_csv(f"../../preds{data}/strainselect/StrainSelect21_vertices.tab.txt", sep= "\t").drop(columns=["StrainSelectID_2019"])
vertices["vertex_dot"] = vertices.vertex.str.replace("_", ".").str.lower()
edges = pd.read_csv(f"../../preds{data}/strainselect/StrainSelect21_edges.tab.txt", sep= "\t")

/scratch/slurm_tmpdir/job_23803810/ipykernel_348609/496185300.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  vertices = pd.read_csv(f"../../preds{data}/strainselect/StrainSelect21_vertices.tab.txt", sep= "\t").drop(columns=["StrainSelectID_2019"])


- How many strains do I lose if I only use matched strains?
- How many assemblies do I lose if I only use linked assembly accessions?

- run partial_ratio and extract the 100 where len(string) > x and remove from the pool.
	- after, split target by dots, and do full match again, if 100 for one, then match it
- run again with the WRatio and extract maybe > 97 or 98

## Merge

In [4]:
vertices_noass = vertices[(vertices.vertex_type.str.endswith("_assembly") == False)&(vertices.vertex_type != "gold_org")&(vertices.vertex_type != "patric_genome")&(vertices.vertex_type != "kegg_genome")&(vertices.vertex.str.contains("GCF_") == False)&(vertices.vertex.str.contains("GCA_") == False)].vertex_dot.to_list() # Remove assembly accessions 

In [5]:
df["vertex_dot"] = df.word_strain_qc.str.replace(" ",".").str.replace("-",".").str.replace("atcc","atcc.").str.replace("dsm","dsm.").str.replace("=","").str.replace("“","").str.replace("”","").str.replace('"',"").str.replace("^","").str.replace(":",".").str.replace("®","").str.replace("™","").str.replace("’","").str.replace("‘","").str.replace("(","").str.replace(")","").str.replace(",","").str.replace("/",".").str.replace("_",".").str.replace("cip","cip.").str.replace("nccp","nccp.").str.replace("...",".").str.replace("..",".")

In [6]:
strains = df.vertex_dot.unique()

In [7]:
strains = [strain for strain in strains if len(strain.replace(".","")) > 2]

## Builtin-parallel
### Small

In [23]:
strains_small = strains[:20000]

In [7]:
all_matches_partial = process.cdist(strains_small, vertices_noass, scorer=fuzz.partial_ratio, workers=150, score_cutoff=95)

In [8]:
all_matches_partial_sparse = sparse.csr_matrix(all_matches_partial)

In [9]:
nonzero_values = all_matches_partial_sparse.data
nonzero_row_indices = all_matches_partial_sparse.nonzero()[0]
nonzero_col_indices = all_matches_partial_sparse.nonzero()[1]

In [10]:
df_all_matches_partial = pd.DataFrame({
    'strain': nonzero_row_indices,
    'strainselect': nonzero_col_indices,
    'score_partial': nonzero_values
})

In [11]:
df_all_matches_partial['strain'] = df_all_matches_partial['strain'].map(lambda x: strains_small[x])
df_all_matches_partial['strainselect'] = df_all_matches_partial['strainselect'].map(lambda x: vertices_noass[x])

In [12]:
def all_combinations(text):
  """
  Generates all possible combinations of splits of a string by period.

  Args:
      text: The string to split.

  Returns:
      A generator object containing all possible combinations of splits.
  """
  for length in range(len(text) + 1):
    for combo in itertools.combinations(text.split('.'), length):
      yield '.'.join(combo)


In [13]:
high_abundant = df_all_matches_partial.groupby("strain").count().sort_values("score_partial", ascending=False).query("score_partial < 2000").index.to_list()

In [14]:
filtered_df_all_matches_partial = df_all_matches_partial.query("strain in @high_abundant")

In [15]:
matches_partial = []
for i in tqdm(filtered_df_all_matches_partial.iterrows(), total=filtered_df_all_matches_partial.shape[0]):
	r = process.extract(i[1]["strain"], list(all_combinations(i[1]["strainselect"]))[1:])
	filtered_df_all_matches_partial.loc[i[0],"score_parts"] = r[0][1]


  0%|          | 0/903492 [00:00<?, ?it/s]

/tmp/ipykernel_183866/555690296.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_all_matches_partial.loc[i[0],"score_parts"] = r[0][1]


In [16]:
filtered_df_all_matches_partial = filtered_df_all_matches_partial.merge(vertices, left_on="strainselect", right_on="vertex_dot")

In [17]:
def filter_strains(df):
    def check_strainselectid(group):
        return len(group['StrainSelectID'].unique()) == 1

    filtered = df.groupby('strain').filter(check_strainselectid)

    # If you need to keep only the first row of remaining strains:
    return filtered.groupby('strain').head(1)

In [18]:
def g(df):
    return df[df.groupby('strain')['score_partial'].transform(max) == df['score_partial']]

df_matches_partial = filter_strains(g(filtered_df_all_matches_partial))

/tmp/ipykernel_183866/2062716169.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  return df[df.groupby('strain')['score_partial'].transform(max) == df['score_partial']]


In [19]:
df_matches_partial = df_matches_partial[~((df_matches_partial['score_parts'] == 90) & (df_matches_partial['score_partial'] == 95))]


In [20]:
df_matches_partial

,strain,strainselect,score_partial,score_parts,vertex,vertex_type,StrainSelectID,vertex_dot
0,ltyr.11z,pantoea.alhagi.ltyr.11z,100.000000,100.000000,pantoea.alhagi.ltyr.11z,gss,t__242713,pantoea.alhagi.ltyr.11z
1,gbl13,acidipropionibacterium.damnosum.igbl13,100.000000,90.909091,acidipropionibacterium.damnosum.igbl13,gss,t__299550,acidipropionibacterium.damnosum.igbl13
4568,dst69,delftia.acidovorans.dst69,100.000000,100.000000,delftia.acidovorans.dst69,gss,t__416547,delftia.acidovorans.dst69
4583,bfr.ca.14430,campylobacter.jejuni.bfr.ca.14430,100.000000,100.000000,campylobacter.jejuni.bfr.ca.14430,gss,t__336881,campylobacter.jejuni.bfr.ca.14430
4587,pseudomonas.alcaligenes.ot69,pseudomonas.alcaligenes.ot.69,96.428574,98.245614,pseudomonas.alcaligenes.ot.69,gss,t__155309,pseudomonas.alcaligenes.ot.69
...,...,...,...,...,...,...,...,...
903443,28isp2.,micromonospora.sp.28isp2.46,100.000000,92.307692,micromonospora.sp.28isp2.46,gss,t__321237,micromonospora.sp.28isp2.46
903444,at1b,exiguobacterium.sp.at1b,100.000000,100.000000,exiguobacterium.sp.at1b,gss,t__1680,exiguobacterium.sp.at1b
903447,sinorhizobium.fredii.gr64,sinorhizobium.fredii.gr64,100.000000,100.000000,sinorhizobium.fredii.gr64,gss,t__123868,sinorhizobium.fredii.gr64
903457,p.angustum.s14,photobacterium.angustum.s14,96.296295,92.307692,photobacterium.angustum.s14,gss,t__129723,photobacterium.angustum.s14


### Full

In [21]:
strains_left = list(set(strains_small) - set(df_matches_partial.strain.unique()))

In [22]:
all_matches_full = process.cdist(strains_left, vertices_noass, workers=150, score_cutoff=90)

In [23]:
all_matches_full_sparse = sparse.csr_matrix(all_matches_full)

In [24]:
nonzero_values = all_matches_full_sparse.data
nonzero_row_indices = all_matches_full_sparse.nonzero()[0]
nonzero_col_indices = all_matches_full_sparse.nonzero()[1]

In [25]:
df_all_matches_full = pd.DataFrame({
    'strain': nonzero_row_indices,
    'strainselect': nonzero_col_indices,
    'score_full': nonzero_values
})

In [26]:
df_all_matches_full['strain'] = df_all_matches_full['strain'].map(lambda x: strains_left[x])
df_all_matches_full['strainselect'] = df_all_matches_full['strainselect'].map(lambda x: vertices_noass[x])

In [27]:
filtered_df_all_matches_full = df_all_matches_full.merge(vertices, left_on="strainselect", right_on="vertex_dot")

In [28]:
def gf(df):
    return df[df.groupby('strain')['score_full'].transform(max) == df['score_full']]
df_matches_full = filter_strains(gf(filtered_df_all_matches_full))

/tmp/ipykernel_183866/1765831059.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  return df[df.groupby('strain')['score_full'].transform(max) == df['score_full']]


In [29]:
df_matches_full

,strain,strainselect,score_full,vertex,vertex_type,StrainSelectID,vertex_dot
43,pseudomonas.st1,pseudomonas.sp.t1,93.750000,pseudomonas.sp.t1,gss,t__533196,pseudomonas.sp.t1
555,enterococcus.faecalis.14,enterococcus.faecalis.14,100.000000,enterococcus.faecalis.14,gss,t__708580,enterococcus.faecalis.14
1346,escherichia.coli.e2,escherichia.coli.e2,100.000000,escherichia.coli.e2,gss,t__164523,escherichia.coli.e2
3155,chryseobacterium.cbo1,chryseobacterium.sp.cbo1,93.333336,chryseobacterium.sp.cbo1,gss,t__210088,chryseobacterium.sp.cbo1
3319,serratia.marcescens.fs14,serratia.marcescens.s14,97.872337,serratia.marcescens.s14,gss,t__550764,serratia.marcescens.s14
...,...,...,...,...,...,...,...
141879,lactiplantibacillus.plantarum.sam2,lactiplantibacillus.plantarum.sm2,98.507462,lactiplantibacillus.plantarum.sm2,gss,t__624280,lactiplantibacillus.plantarum.sm2
142986,thermus.thermophilus.tmy,thermus.thermophilus.tmy,100.000000,thermus.thermophilus.tmy,gss,t__246251,thermus.thermophilus.tmy
143012,bacillus.licheniformis.wj.2,bacillus.licheniformis.w.2,98.113205,bacillus.licheniformis.w.2,gss,t__730803,bacillus.licheniformis.w.2
143770,kctc.15717,kctc 15717,90.000000,KCTC 15717,brc_cat_id,t__189744,kctc 15717


### Join all matches

In [30]:
matches = pd.concat([df_matches_full, df_matches_partial])
final = df.merge(matches,left_on="vertex_dot", right_on="strain", how="left")

In [31]:
matches

,strain,strainselect,score_full,vertex,vertex_type,StrainSelectID,vertex_dot,score_partial,score_parts
43,pseudomonas.st1,pseudomonas.sp.t1,93.750000,pseudomonas.sp.t1,gss,t__533196,pseudomonas.sp.t1,NaN,NaN
555,enterococcus.faecalis.14,enterococcus.faecalis.14,100.000000,enterococcus.faecalis.14,gss,t__708580,enterococcus.faecalis.14,NaN,NaN
1346,escherichia.coli.e2,escherichia.coli.e2,100.000000,escherichia.coli.e2,gss,t__164523,escherichia.coli.e2,NaN,NaN
3155,chryseobacterium.cbo1,chryseobacterium.sp.cbo1,93.333336,chryseobacterium.sp.cbo1,gss,t__210088,chryseobacterium.sp.cbo1,NaN,NaN
3319,serratia.marcescens.fs14,serratia.marcescens.s14,97.872337,serratia.marcescens.s14,gss,t__550764,serratia.marcescens.s14,NaN,NaN
...,...,...,...,...,...,...,...,...,...
903443,28isp2.,micromonospora.sp.28isp2.46,NaN,micromonospora.sp.28isp2.46,gss,t__321237,micromonospora.sp.28isp2.46,100.000000,92.307692
903444,at1b,exiguobacterium.sp.at1b,NaN,exiguobacterium.sp.at1b,gss,t__1680,exiguobacterium.sp.at1b,100.000000,100.000000
903447,sinorhizobium.fredii.gr64,sinorhizobium.fredii.gr64,NaN,sinorhizobium.fredii.gr64,gss,t__123868,sinorhizobium.fredii.gr64,100.000000,100.000000
903457,p.angustum.s14,photobacterium.angustum.s14,NaN,photobacterium.angustum.s14,gss,t__129723,photobacterium.angustum.s14,96.296295,92.307692


---

In [25]:
# Define the number of processes to run in parallel
num_processes = 150

# Split the strains into batches
batch_size = 100

Add abbreviations to the names of the strains, and then merge the two datasets.

In [ ]:
# ss_v_noass["vertex_abb"] = ss_v_noass.vertex_dot.str.replace(r"^([a-z])\w+\.", r"\1.",regex=True)
# ss_v_noass_abb = pd.concat([ss_v_noass.iloc[:,:-1],
# 		   ss_v_noass.loc[:,["vertex","vertex_type","StrainSelectID","StrainSelectID_2019","vertex_abb"]].rename(columns={"vertex_abb":"vertex_dot"})
# 		   ]
# 		   ,axis=0).drop_duplicates()

## Small

In [ ]:
# Split the strains into batches
strain_batches = [strains[i:i+batch_size] for i in range(0, len(strains), batch_size)]

# Define a function to process each batch of strains
def process_strain_batch(batch):
	matches = []
	for s in batch:
		result = process.extract(s, ss_v_noass.vertex_dot, limit=1,scorer=fuzz.partial_ratio)
		matches.append([s,result])
	return matches

# Create a pool of processes
pool = multiprocessing.Pool(processes=num_processes)

# Apply the function to each batch of strains in parallel
results = pool.map(process_strain_batch, strain_batches)

# Close the pool of processes
pool.close()
pool.join()

# Combine the results from all batches
all_matches_partial = []
for result in results:
	all_matches_partial.extend(result)


In [ ]:
df_all_matches_partial = pd.concat([pd.DataFrame(all_matches_partial)[0],pd.DataFrame(all_matches_partial)[1].apply(pd.Series)[0].apply(pd.Series)],axis=1)

In [ ]:
def all_combinations(text):
  """
  Generates all possible combinations of splits of a string by period.

  Args:
      text: The string to split.

  Returns:
      A generator object containing all possible combinations of splits.
  """
  for length in range(len(text) + 1):
    for combo in itertools.combinations(text.split('.'), length):
      yield '.'.join(combo)


In [ ]:
perf = [i for i in all_matches_partial if i[1][0][1]>95]
matches_partial = []
for i in perf:
	r = process.extract(i[0], list(all_combinations(i[1][0][0]))[1:])
	if r[0][1] > 95:
		matches_partial.append([i[0], i[1][0][0], i[1][0][1], r[0][1]])

In [ ]:
df_matches_partial = pd.DataFrame(
	matches_partial, columns=["strain", "strainselect","score_partial", "score_parts"]
)

In [ ]:
# Merge the matches with the original DataFrame
df_partial = df.merge(
	df_matches_partial, left_on="vertex_dot", right_on="strain", how="left"
)

In [ ]:
strains_left = df_partial[
	df_partial.strainselect.isnull()
].vertex_dot.unique()

## Full

In [ ]:
# Run full match on remaining strains
matches_full = []

# Split the strains into batches
strain_batches = [strains_left[i:i+batch_size] for i in range(0, len(strains_left), batch_size)]

# Define a function to process each batch of strains
def process_strain_batch(batch):
	matches = []
	for s in batch:
		result = process.extract(s, ss_v_noass.vertex.str.lower(), limit=1)
		matches.append([s,result])
	return matches

# Create a pool of processes
pool = multiprocessing.Pool(processes=num_processes)

# Apply the function to each batch of strains in parallel
results = pool.map(process_strain_batch, strain_batches)

# Close the pool of processes
pool.close()
pool.join()

# Combine the results from all batches
all_matches_full = []
for result in results:
	all_matches_full.extend(result)

In [ ]:
df_all_matches_full = pd.concat([pd.DataFrame(all_matches_full)[0],pd.DataFrame(all_matches_full)[1].apply(pd.Series)[0].apply(pd.Series)],axis=1)

In [ ]:
matches_full = [[i[0],i[1][0][0],i[1][0][1]] for i in all_matches_full if i[1][0][1]>97.6]


In [ ]:
df_matches_full = pd.DataFrame(
	matches_full, columns=["strain", "strainselect","score_full"]
)

In [ ]:
merged = df.merge(
	pd.concat([df_matches_partial,df_matches_full]), left_on="vertex_dot", right_on="strain", how="left"
)

In [ ]:
merged.drop(columns=["strain"],inplace=True)

In [ ]:
vertex_merged = merged.merge(ss_v, left_on="strainselect", right_on="vertex_dot", how="left")

In [ ]:
selected = ss_v[ss_v["StrainSelectID"].isin(vertex_merged.dropna(subset = "strainselect")["StrainSelectID"].unique())]

In [ ]:
selected.StrainSelectID.unique().shape

In [ ]:
len(selected[selected.vertex.str.contains("GCA")].StrainSelectID.unique())

In [ ]:
len(selected[selected.vertex.str.contains("GCA")])

---

# As done on script

In [8]:
strains_small = strains[:4000]
batch_size = 2000
strain_batches = [strains_small[i:i + batch_size] for i in range(0, len(strains_small), batch_size)]

In [ ]:
%%time
all_matches_partial = process.cdist(strain_batches[0], 
                                    vertices_noass, scorer=fuzz.partial_ratio, 
                                    workers=40, score_cutoff=95)

In [9]:
def all_combinations(text):
    for length in range(len(text) + 1):
        for combo in itertools.combinations(text.split('.'), length):
            yield '.'.join(combo)

def filter_strains(df):
    def check_strainselectid(group):
        return len(group['StrainSelectID'].unique()) == 1

    filtered = df.groupby('strain').filter(check_strainselectid)

    return filtered.groupby('strain').head(1)

def g(df):
        return df[df.groupby('strain')['score_partial'].transform("max") == df['score_partial']]

def gf(df):
    return df[df.groupby('strain')['score_full'].transform("max") == df['score_full']]

In [ ]:
workers = 40
batch_results = []

for strain_batch in tqdm(strain_batches):
    # Partial matches
    all_matches_partial = process.cdist(strain_batch, vertices_noass, scorer=fuzz.partial_ratio, workers=workers, score_cutoff=95)
    all_matches_partial_sparse = sparse.csr_matrix(all_matches_partial)

    nonzero_row_indices, nonzero_col_indices = all_matches_partial_sparse.nonzero()
    nonzero_values = all_matches_partial_sparse.data

    df_all_matches_partial = pd.DataFrame({
        'strain': nonzero_row_indices,
        'strainselect': nonzero_col_indices,
        'score_partial': nonzero_values
    })

    df_all_matches_partial['strain'] = df_all_matches_partial['strain'].map(lambda x: strain_batch[x])
    df_all_matches_partial['strainselect'] = df_all_matches_partial['strainselect'].map(lambda x: vertices_noass[x])

    high_abundant = df_all_matches_partial.groupby("strain").size().sort_values(ascending=False)
    high_abundant = high_abundant[high_abundant < 2000].index.to_list()
    filtered_df_all_matches_partial = df_all_matches_partial[df_all_matches_partial['strain'].isin(high_abundant)].copy()

    filtered_df_all_matches_partial.loc[:, 'score_parts'] = filtered_df_all_matches_partial.apply(
        lambda row: process.extractOne(row['strain'], list(all_combinations(row['strainselect']))[1:],score_cutoff=80)[1], axis=1
    )

    filtered_df_all_matches_partial = filtered_df_all_matches_partial.merge(vertices, left_on="strainselect", right_on="vertex_dot")

    df_matches_partial = filter_strains(g(filtered_df_all_matches_partial))
    df_matches_partial = df_matches_partial[df_matches_partial['score_partial'] > 60]
    df_matches_partial = df_matches_partial[~((df_matches_partial['score_parts'] <= 90) & (df_matches_partial['vertex_type'] == "biocyc_pgdb"))]

    # Full matches
    strains_left = list(set(strain_batch) - set(df_matches_partial['strain'].unique()))

    all_matches_full = process.cdist(strains_left, vertices_noass, workers=workers, score_cutoff=90)
    all_matches_full_sparse = sparse.csr_matrix(all_matches_full)
    nonzero_row_indices, nonzero_col_indices = all_matches_full_sparse.nonzero()
    nonzero_values = all_matches_full_sparse.data

    df_all_matches_full = pd.DataFrame({
        'strain': nonzero_row_indices,
        'strainselect': nonzero_col_indices,
        'score_full': nonzero_values
    })

    df_all_matches_full['strain'] = df_all_matches_full['strain'].map(lambda x: strains_left[x])
    df_all_matches_full['strainselect'] = df_all_matches_full['strainselect'].map(lambda x: vertices_noass[x])

    filtered_df_all_matches_full = df_all_matches_full.merge(vertices, left_on="strainselect", right_on="vertex_dot")

    df_matches_full = filter_strains(gf(filtered_df_all_matches_full))

    matches = pd.concat([df_matches_full, df_matches_partial], ignore_index=True)

    batch_results.append(matches)
 

 50%|█████     | 1/2 [03:03<03:03, 183.16s/it]

In [27]:
pd.concat(batch_results)

,strain,strainselect,score_full,vertex,vertex_type,StrainSelectID,vertex_dot,score_partial,score_parts
0,pseudomonas.savastanoi.pv,pseudomonas.savastanoi.psv,98.039215,pseudomonas.savastanoi.psv,gss,t__120012,pseudomonas.savastanoi.psv,NaN,NaN
1,arthrobacter.sp.at.5,arthrobacter.sp.at5,97.435898,arthrobacter.sp.at5,gss,t__507092,arthrobacter.sp.at5,NaN,NaN
2,shigella.flexneri.snt22,shigella.flexneri.s22,95.454544,shigella.flexneri.s22,gss,t__396768,shigella.flexneri.s22,NaN,NaN
3,streptomyces.sp.h3.2,streptomyces.sp.h3.2,100.000000,streptomyces.sp.h3.2,gss,t__678025,streptomyces.sp.h3.2,NaN,NaN
4,bacillus.cereus.rsa1,bacillus.cereus.rsa21,97.560974,bacillus.cereus.rsa21,gss,t__612834,bacillus.cereus.rsa21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
440,e.coli.bw25113,escherichia.coli.bw25113,NaN,escherichia.coli.bw25113,gss,t__142594,escherichia.coli.bw25113,96.296295,92.307692
441,d.pigrum.ambr11,dolosigranulum.pigrum.ambr11,NaN,dolosigranulum.pigrum.ambr11,gss,t__465392,dolosigranulum.pigrum.ambr11,96.551727,92.857143
442,l.casei.ambr2,lactobacillus.casei.ambr2,NaN,lactobacillus.casei.ambr2,gss,t__192251,lactobacillus.casei.ambr2,96.000000,91.666667
443,rhodovulum.sulfidophilum.kkmi01,rhodovulum.sulfidophilum.kkmi01,NaN,rhodovulum.sulfidophilum.kkmi01,gss,t__451137,rhodovulum.sulfidophilum.kkmi01,100.000000,100.000000


In [ ]:
final = df.merge(pd.concat(batch_results),left_on="vertex_dot", right_on="strain", how="left")
final = final.drop(columns = ["vertex_dot_x", "vertex_dot_y","strainselect","strain"])
final.rename(columns={"score":"ner_score","vertex":"strainselect_vertex"})

## Run by parts to find slow 

In [37]:
import dask.array as da
from scipy.sparse import csr_matrix

In [34]:
workers = 40
batch_results = []
strain_batch = strain_batches[0]

### Partial matches

In [35]:
%%time
all_matches_partial = process.cdist(strain_batch, vertices_noass, scorer=fuzz.partial_ratio, workers=workers, score_cutoff=95)

CPU times: user 1h 42min 22s, sys: 37.7 s, total: 1h 43min
Wall time: 2min 39s


In [41]:
all_matches_partial.shape

(2000, 3014102)

In [ ]:
# Function to convert a chunk to a sparse CSR matrix
def to_sparse(chunk):
    return csr_matrix(chunk)

In [76]:
%%time

# Convert the numpy array to a Dask array
#chunk_size_columns = 50000 
dask_array = da.from_array(all_matches_partial, chunks=(all_matches_partial.shape[0], all_matches_partial.shape[1]//(workers - 1)))

# Apply the conversion function to each chunk
sparse_matrix_dask = dask_array.map_blocks(to_sparse, dtype=csr_matrix)

# Compute the result
sparse_matrix = sparse_matrix_dask.compute()


CPU times: user 2min 16s, sys: 4.83 s, total: 2min 21s
Wall time: 25.6 s


In [77]:
%%time
all_matches_partial_sparse = sparse.csr_matrix(all_matches_partial)

CPU times: user 36.2 s, sys: 533 ms, total: 36.7 s
Wall time: 37.1 s


In [48]:
%%time
nonzero_row_indices, nonzero_col_indices = sparse_matrix.nonzero()
nonzero_values = all_matches_partial_sparse.data
df_all_matches_partial = pd.DataFrame({
    'strain': nonzero_row_indices,
    'strainselect': nonzero_col_indices,
    'score_partial': nonzero_values
})

CPU times: user 29.2 ms, sys: 40.9 ms, total: 70 ms
Wall time: 207 ms


In [49]:
%%time
df_all_matches_partial['strain'] = df_all_matches_partial['strain'].map(lambda x: strain_batch[x])
df_all_matches_partial['strainselect'] = df_all_matches_partial['strainselect'].map(lambda x: vertices_noass[x])
high_abundant = df_all_matches_partial.groupby("strain").size().sort_values(ascending=False)
high_abundant = high_abundant[high_abundant < 2000].index.to_list()
filtered_df_all_matches_partial = df_all_matches_partial[df_all_matches_partial['strain'].isin(high_abundant)].copy()

CPU times: user 457 ms, sys: 45.9 ms, total: 503 ms
Wall time: 574 ms


In [50]:
%%time
filtered_df_all_matches_partial.loc[:, 'score_parts'] = filtered_df_all_matches_partial.apply(
    lambda row: process.extractOne(row['strain'], list(all_combinations(row['strainselect']))[1:],score_cutoff=80)[1], axis=1
)

CPU times: user 2.44 s, sys: 16 ms, total: 2.46 s
Wall time: 2.47 s


In [51]:
%%time
filtered_df_all_matches_partial = filtered_df_all_matches_partial.merge(vertices, left_on="strainselect", right_on="vertex_dot")

CPU times: user 2.15 s, sys: 25 ms, total: 2.18 s
Wall time: 2.19 s


In [52]:
%%time
df_matches_partial = filter_strains(g(filtered_df_all_matches_partial))
df_matches_partial = df_matches_partial[df_matches_partial['score_partial'] > 60]
df_matches_partial = df_matches_partial[~((df_matches_partial['score_parts'] <= 90) & (df_matches_partial['vertex_type'] == "biocyc_pgdb"))]

CPU times: user 104 ms, sys: 7.95 ms, total: 112 ms
Wall time: 123 ms


### Full matches

In [53]:
%%time
strains_left = list(set(strain_batch) - set(df_matches_partial['strain'].unique()))

CPU times: user 790 μs, sys: 0 ns, total: 790 μs
Wall time: 740 μs


In [54]:
%%time
all_matches_full = process.cdist(strains_left, vertices_noass, workers=workers, score_cutoff=90)

CPU times: user 1min 20s, sys: 18.4 s, total: 1min 39s
Wall time: 4.06 s


In [56]:
all_matches_full.shape

(1472, 3014102)

In [58]:
all_matches_full.shape[0]

1472

In [63]:
def to_sparse(chunk):
    return csr_matrix(chunk)

In [68]:
all_matches_full.shape[1]//39

77284

In [78]:
%%time

# Convert the numpy array to a Dask array
dask_array = da.from_array(all_matches_full, chunks=(all_matches_full.shape[0], all_matches_full.shape[1]//(workers -1 )))

# Apply the conversion function to each chunk
sparse_matrix_dask = dask_array.map_blocks(to_sparse, dtype=csr_matrix)

# Compute the result
full_sparse_matrix = sparse_matrix_dask.compute()


CPU times: user 1min 38s, sys: 4 s, total: 1min 42s
Wall time: 23.5 s


In [59]:
%%time
all_matches_full_sparse = sparse.csr_matrix(all_matches_full)

CPU times: user 26.6 s, sys: 21 ms, total: 26.7 s
Wall time: 26.7 s


In [29]:
%%time
nonzero_row_indices, nonzero_col_indices = all_matches_full_sparse.nonzero()
nonzero_values = all_matches_full_sparse.data

df_all_matches_full = pd.DataFrame({
    'strain': nonzero_row_indices,
    'strainselect': nonzero_col_indices,
    'score_full': nonzero_values
})

CPU times: user 1.18 ms, sys: 2.99 ms, total: 4.17 ms
Wall time: 9.1 ms


In [30]:
%%time
df_all_matches_full['strain'] = df_all_matches_full['strain'].map(lambda x: strains_left[x])
df_all_matches_full['strainselect'] = df_all_matches_full['strainselect'].map(lambda x: vertices_noass[x])

filtered_df_all_matches_full = df_all_matches_full.merge(vertices, left_on="strainselect", right_on="vertex_dot")

df_matches_full = filter_strains(gf(filtered_df_all_matches_full))

CPU times: user 1.98 s, sys: 82.7 ms, total: 2.06 s
Wall time: 2.25 s


In [31]:
%%time
matches = pd.concat([df_matches_full, df_matches_partial], ignore_index=True)

batch_results.append(matches)

CPU times: user 2.7 ms, sys: 0 ns, total: 2.7 ms
Wall time: 3.45 ms


# Polars

In [6]:
import polars as pl
from tqdm.autonotebook import tqdm
from rapidfuzz import fuzz
from rapidfuzz import process
import multiprocessing
import itertools
from scipy import sparse

data = "920"

df = pl.read_parquet(f"../../preds{data}/REL_output/preds.pqt")
vertices = pl.read_csv(f"../../preds{data}/strainselect/StrainSelect21_vertices.tab.txt", separator="\t").drop("StrainSelectID_2019")
vertices = vertices.with_columns(pl.col("vertex").str.replace("_", ".").str.to_lowercase().alias("vertex_dot"))
edges = pl.read_csv(f"../../preds{data}/strainselect/StrainSelect21_edges.tab.txt", separator="\t")



In [8]:
vertices_noass = vertices.filter(
    (~pl.col("vertex_type").str.contains("_assembly")) &
    (pl.col("vertex_type") != "gold_org") &
    (pl.col("vertex_type") != "patric_genome") &
    (pl.col("vertex_type") != "kegg_genome") &
    (~pl.col("vertex").str.contains("GCF_")) &
    (~pl.col("vertex").str.contains("GCA_"))
)["vertex_dot"].to_list()

In [11]:
df = df.with_columns(
    pl.col("word_strain_qc")
    .str.replace(r" ", r".")
    .str.replace(r"-", r".")
    .str.replace(r"atcc", r"atcc.")
    .str.replace(r"dsm", r"dsm.")
    .str.replace(r"=", r"")
    .str.replace(r"“", r"")
    .str.replace(r"”", r"")
    .str.replace(r'"', r"")
    .str.replace(r"\^", r"")
    .str.replace(r":", r".")
    .str.replace(r"®", r"")
    .str.replace(r"™", r"")
    .str.replace(r"’", r"")
    .str.replace(r"‘", r"")
    .str.replace(r"\(", r"")
    .str.replace(r"\)", r"")
    .str.replace(r",", r"")
    .str.replace(r"/", r".")
    .str.replace(r"_", r".")
    .str.replace(r"cip", r"cip.")
    .str.replace(r"nccp", r"nccp.")
    .str.replace(r"\.\.\.", r".")
    .str.replace(r"\.\.", r".")
    .alias("vertex_dot")
)

In [30]:
strains = df["vertex_dot"].unique().to_list()
strains = [strain for strain in strains if len(strain.replace(".", "")) > 2]

In [42]:
strains_small = strains[:3000]
batch_size = 1000
strain_batches = [strains_small[i:i + batch_size] for i in range(0, len(strains_small), batch_size)]


In [34]:
def all_combinations(text):
    for length in range(len(text) + 1):
        for combo in itertools.combinations(text.split('.'), length):
            yield '.'.join(combo)

def filter_strains(df):
    def check_strainselectid(group):
        return len(group["StrainSelectID"].unique()) == 1

    filtered = df.groupby("strain").filter(check_strainselectid)
    return filtered.groupby("strain").first()

def g(df):
    return df.filter(pl.col("score_partial") == df.groupby("strain")["score_partial"].max())

def gf(df):
    return df.filter(pl.col("score_full") == df.groupby("strain")["score_full"].max())


In [35]:
workers = 40
batch_results = []

In [ ]:
for strain_batch in tqdm(strain_batches):
    # Partial matches
    all_matches_partial = process.cdist(strain_batch, vertices_noass, scorer=fuzz.partial_ratio, workers=workers, score_cutoff=95)
    all_matches_partial_sparse = sparse.csr_matrix(all_matches_partial)

    nonzero_row_indices, nonzero_col_indices = all_matches_partial_sparse.nonzero()
    nonzero_values = all_matches_partial_sparse.data

    df_all_matches_partial = pl.DataFrame({
        "strain": nonzero_row_indices,
        "strainselect": nonzero_col_indices,
        "score_partial": nonzero_values
    })

    df_all_matches_partial = df_all_matches_partial.with_columns([
        pl.col("strain").apply(lambda x: strain_batch[x]),
        pl.col("strainselect").apply(lambda x: vertices_noass[x])
    ])

    high_abundant = df_all_matches_partial.groupby("strain").agg(pl.count().alias("count")).sort("count", descending=True)
    high_abundant = high_abundant.filter(pl.col("count") < 2000)["strain"].to_list()
    filtered_df_all_matches_partial = df_all_matches_partial.filter(pl.col("strain").is_in(high_abundant)).with_columns(
        pl.col("strainselect").apply(lambda strainselect: process.extractOne(strainselect, list(all_combinations(strainselect))[1:], score_cutoff=80)[1]).alias("score_parts")
    )

    filtered_df_all_matches_partial = filtered_df_all_matches_partial.join(vertices, left_on="strainselect", right_on="vertex_dot")

    df_matches_partial = filter_strains(g(filtered_df_all_matches_partial))
    df_matches_partial = df_matches_partial.filter(pl.col("score_partial") > 60)
    df_matches_partial = df_matches_partial.filter(~((pl.col("score_parts") <= 90) & (pl.col("vertex_type") == "biocyc_pgdb")))

    # Full matches
    strains_left = list(set(strain_batch) - set(df_matches_partial["strain"].unique()))

    all_matches_full = process.cdist(strains_left, vertices_noass, workers=workers, score_cutoff=90)
    all_matches_full_sparse = sparse.csr_matrix(all_matches_full)
    nonzero_row_indices, nonzero_col_indices = all_matches_full_sparse.nonzero()
    nonzero_values = all_matches_full_sparse.data

    df_all_matches_full = pl.DataFrame({
        "strain": nonzero_row_indices,
        "strainselect": nonzero_col_indices,
        "score_full": nonzero_values
    })

    df_all_matches_full = df_all_matches_full.with_columns([
        pl.col("strain").apply(lambda x: strains_left[x]),
        pl.col("strainselect").apply(lambda x: vertices_noass[x])
    ])

    filtered_df_all_matches_full = df_all_matches_full.join(vertices, left_on="strainselect", right_on="vertex_dot")

    df_matches_full = filter_strains(gf(filtered_df_all_matches_full))

    matches = pl.concat([df_matches_full, df_matches_partial])

    batch_results.append(matches)

  0%|          | 0/3 [00:00<?, ?it/s]